### Run ACTION_RECOGNITION.ipynb before running model_analysis.ipynb #####

In [ ]:
from sklearn import metrics
import os
import pickle
import matplotlib.pyplot as plt
import collections
from movingpose.estimator import classifiers
from movingpose.estimator import neighbors

### Tuning parameters

In [1]:
# Strength of precision vs recall
beta = 1

NameError: name 'beta' is not defined

### Load model data and generate metrics

In [ ]:
cur_uuid = str(input("cur_uuid:"))
dir_path = f"../pickle/{cur_uuid}"

if not (cur_uuid != "" and os.path.isdir(dir_path)):
    print("ERROR :: UUID not found")
    print("Please enter the correct UUID or run action_recognition.ipynb and pass in the associated UUID")
    exit(1)

print("Loading data...")

with open(dir_path + "/train_test_splits.p", 'rb') as fp:
    training_data = pickle.load(fp)

X_train, X_test = training_data["X_train"], training_data["X_test"]
y_train, y_test = training_data["y_train"], training_data["y_test"]

all_predictions_fns = [file_name for file_name in os.listdir(dir_path) if file_name.startswith("prediction")]

# results_dict[str(action_classifier)] = {
#   "y_pred": y_pred,
#   "action_classifier_params": action_classifier_params,
#   "confusion_matrix": confusion_matrix,
#   "precision": precision,
#   "recall": recall,
#   "fbetascore": fscore,
#   "support": support,
#   "accuracy": accuracy,
#   "prediction_speed": prediction_speed
# }
predictions_dict = {}

for prediction_fn in all_predictions_fns:
    with open(f"{dir_path}/{prediction_fn}.p", 'rb') as fp:
        prediction_info = pickle.load(fp)

        action_classifier_name = prediction_info["action_classifier_name"]
        y_pred = prediction_info["y_pred"]
        action_classifier_params = prediction_info["action_classifier_params"]
        prediction_speed = prediction_info["prediction_speed"]

        confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
        precision, recall, fscore, support = metrics.precision_recall_fscore_support(y_test, y_pred, beta=beta)
        accuracy = metrics.accuracy_score(y_test, y_pred)

        predictions_dict[action_classifier_name] = {
            "y_pred": y_pred,
            "action_classifier_params": action_classifier_params,
            "confusion_matrix": confusion_matrix,
            "precision": precision,
            "recall": recall,
            "fbetascore": fscore,
            "support": support,
            "accuracy": accuracy,
            "prediction_speed": prediction_speed
        }

#### Organize hyper parameter combinations

In [ ]:
# hpc_values[hpc_name] = set(hpc_value, ... (all hpc_values))
hpc_values_dict = collections.defaultdict(set)

for model_name, prediction_info in predictions_dict.items():
    action_classifier = prediction_info["action_classifier_params"]
    pose_estimator = action_classifier["nearest_pose_estimator"]

    hpcs = {
        "theta": action_classifier["theta"],
        "n": action_classifier["n"],
        "alpha": pose_estimator["alpha"],
        "beta": pose_estimator["beta"],
        "kappa": pose_estimator["kappa"],
        "n_neighbors": pose_estimator["n_neighbors"],
        "n_training_neighbors": pose_estimator["n_training_neighbors"]
    }

    for hpc_name, hpc_value in hpcs.items():
        hpc_values_dict[hpc_name].add(hpc_value)

# hpc_combinations[hpc_name] = set("str(action_recognition) with wildcard for hpc_value", ... (all hpc name templates))
hpc_combinations = collections.defaultdict(set)
for hpc_name in hpc_values_dict.keys():
    for theta in hpc_values_dict["theta"]:
        for n in hpc_values_dict["n"]:
            for alpha in hpc_values_dict["alpha"]:
                for beta in hpc_values_dict["beta"]:
                    for kappa in hpc_values_dict["kappa"]:
                        for n_neighbors in hpc_values_dict["n_neighbors"]:
                            for n_training_neighbors in hpc_values_dict["n_training_neighbors"]:
                                nearest_pose_estimator = neighbors.NearestPoses(
                                    n_neighbors=n_neighbors,
                                    n_training_neighbors=n_training_neighbors,
                                    alpha=alpha,
                                    beta=beta,
                                    kappa=kappa
                                )
                                action_classifier = classifiers.ActionClassifier(
                                    nearest_pose_estimator=nearest_pose_estimator,
                                    theta=theta,
                                    n=n
                                )
                                model_name = str(action_classifier)
                                model_name_parts = model_name.split("=")
                                for i, model_name_part in enumerate(model_name_parts):
                                    if not model_name_part.ends_with(hpc_name):
                                        continue
                                    model_name_parts[i+1] = "%d"
                                    break
                                template_name = "=".join(model_name_parts)
                                hpc_combinations[hpc_name].add(template_name)

#### Assert all required prediction files exist

In [ ]:
for hpc_name, model_name_templates in hpc_combinations.items():
    for hpc_value in hpc_values_dict.values():
        for model_name_template in model_name_templates:
            assert (os.path.isfile(f"{dir_path}/{model_name_template % hpc_value}.p"),
                        f"{dir_path}/{model_name_template % hpc_value}.p file does not exist")

#### Visualize hyperparameter combinations

In [ ]:
metrics = [
    {
        "id": "prediction_speed",
        "name": "Time to Predict 80 Actions (min)"
    },
    {
        "id": "accuracy",
        "name": "Accuracy (%)"
    },
    {
        "id": "fbetascore",
        "name": f"F{beta}-Score (%)"
    },
    {
        "id": "precision",
        "name": "Precision (%)"
    },
    {
        "id": "recall",
        "name": "Recall (%)"
    }
]
hpc_names = list(hpc_values_dict.copy())

plots = collections.defaultdict(list)
for hpc_name, model_name_templates in hpc_combinations.items():
    for hpc_num, hpc_value in enumerate(hpc_values_dict.values()):
        for model_name_template in model_name_templates:
            prediction_info = predictions_dict[model_name_template % hpc_value]
            for metric_num, metric_info in enumerate(metrics):
                plots[(hpc_num, metric_num)].append((hpc_value, prediction_info[metric_info["id"]]))

fig, axs = plt.subplots(len(hpc_values_dict.values()), 5)
for (col, row), plot in plots.items():
    for (x, y) in plot:
        axs[col, row].scatter(x, y)
    xlabel = f"{hpc_names[col].title()} Values"
    ylabel = metrics[row]["name"]
    axs[col, row].set(xlabel=xlabel, ylabel=ylabel)
    axs[col, row].set_title("Change in {ylabel} as {xlabel} Change")

plt.show()
